# Examen Final DIEGO POBLETE

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [1]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [2]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [3]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [4]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [5]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [11]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [7]:
gamma = 1
sigma = .005

In [8]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [13]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

Sólo puedes pedir las preguntas 1 vez.


In [10]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 2:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,049.00 (365 días después).


Pregunta 3:

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es
la misma que en el modelo de Hull-White. Comente.


Pregunta 4:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .1.4756 .


Pregunta 5:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.0%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 6:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
for

## Respuestas

### Pregunta 1

La principal diferencia que existe entre valorizar por medida libre de riesgo y medida forward en el modelo de Hull y White es la manera de calcular el valor presente. 
Por el lado de la medida libre de riesgo, se debe discretizar la trayectoria (tasa el cual se necesita para traer los flujos a valor presente es estocástica)
de tal forma de descontar para cada "t" de la curva 
que corresponde al producto de todos los factores de descuento entre el primer instante hasta el momento anterior al fin del periodo.
todo eso finalmente se multiplica por el payoff a vencimiento. 
mientras que en la medida forward, la manera para obtener el VP es descontando con la tasa del mismo periodo que vence el contrato. 
eso multiplicado al valor esperado del payoff al vencimiento. 

### Pregunta 2

In [19]:
icp_0 = 10000
icp_365 = 10049

t_0 = 0
t_1 = 365

TNA = ((icp_365/icp_0)-1)*360/(t_1-t_0)
print(f'TNA: {TNA:.4%}')

TNA: 0.4833%


### Pregunta 3

La diferencia es que para el caso del Modelo Hull-White obtiene de mejor forma la curva cero cupón de mercado, mientras que el modelo Vasicek vimos en clases que no se ajusta de buena forma. Si bien las variables auxiliares para obtener el valor de la opción también se pueden encontrar en el modelo de Vasicek, estas no arrojaran el mismo valor debido a la dinámica de la tasa
y lo mencionado anteriormente. 

### Pregunta 4

In [32]:
gamma = 1.0
sigma = 0.005
strike = 0.98

strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))
    

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


### Pregunta 5

In [34]:
gamma = 1.0
sigma = 0.005
strike = 0.98
nocional = 1

Call_option = hw.zcb_call_put(
        hw.CallPut.CALL,
        strike,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    )
print(f'valor call: {Call_option:.8}')

valor call: 0.019553576


### Pregunta 6

In [24]:
crv = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear')

t1 = 482
t2 = 654

r1 = crv(t1)
r2 = crv(t2)

df1 = math.exp(-r1*t1/365)
df2 = math.exp(-r2*t2/365)

fwd = (df1 / df2 - 1.0)* 360.0 /(t2 - t1)

print(f'fwd: {fwd:.4%}')


fwd: 0.0411%


### Pregunta 7

In [ ]:
#no alcancé :(
